In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split,cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score,f1_score

In [2]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("mlg-ulb/creditcardfraud")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/creditcardfraud


In [3]:
df = pd.read_csv("/kaggle/input/creditcardfraud/creditcard.csv")
df.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [4]:
df["Class"].value_counts()

,count
Class,
0,284315
1,492


#It appears we have imblanced dataset. I'll try two ways to see which is better.

#First Method: under sampling

In [5]:
no_frd = df[df["Class"]==0]
frd = df[df["Class"]==1]
no_frd_sample = no_frd.sample(n=492)
new_df = pd.concat([no_frd_sample,frd],axis=0)
new_df.head()


,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
196939,131769.0,-1.092041,0.339493,-1.001952,-3.158126,1.021427,-0.660715,1.422697,0.041987,-2.291187,...,0.022317,0.207933,-0.550717,0.131238,1.532327,1.098762,-0.269647,-0.094393,98.13,0
182749,125532.0,2.018978,-0.981888,-1.607440,-0.570586,0.371100,1.483999,-0.803163,0.475355,-0.034754,...,-0.117819,0.454730,0.123886,-0.758215,0.040156,0.021710,0.053237,-0.079384,1.00,0
28340,34962.0,0.856895,-1.167332,0.712375,0.525256,-1.093928,0.880391,-0.872982,0.419408,-0.531845,...,-0.053385,0.098998,-0.129378,-0.335095,0.244636,-0.192948,0.068889,0.042126,168.00,0
168956,119456.0,2.081981,-0.857944,-1.233755,-0.925748,-0.661549,-1.107825,-0.358120,-0.318278,-0.841877,...,0.539428,1.464671,-0.061713,0.156438,0.147153,0.043402,-0.044805,-0.064386,60.85,0
137583,82231.0,1.362154,-0.229830,-1.544174,-1.110608,2.096124,3.107214,-0.398104,0.709341,0.186003,...,-0.142757,-0.409931,-0.121053,1.099061,0.702428,1.137646,-0.093744,-0.014855,3.57,0


In [6]:
new_df["Class"].value_counts()

,count
Class,
0,492
1,492


In [7]:
X = new_df.drop("Class",axis=1)
Y= new_df["Class"]


In [8]:
print(X)


            Time        V1        V2        V3        V4        V5        V6  \
196939  131769.0 -1.092041  0.339493 -1.001952 -3.158126  1.021427 -0.660715   
182749  125532.0  2.018978 -0.981888 -1.607440 -0.570586  0.371100  1.483999   
28340    34962.0  0.856895 -1.167332  0.712375  0.525256 -1.093928  0.880391   
168956  119456.0  2.081981 -0.857944 -1.233755 -0.925748 -0.661549 -1.107825   
137583   82231.0  1.362154 -0.229830 -1.544174 -1.110608  2.096124  3.107214   
...          ...       ...       ...       ...       ...       ...       ...   
279863  169142.0 -1.927883  1.125653 -4.518331  1.749293 -1.566487 -2.010494   
280143  169347.0  1.378559  1.289381 -5.004247  1.411850  0.442581 -1.326536   
280149  169351.0 -0.676143  1.126366 -2.213700  0.468308 -1.120541 -0.003346   
281144  169966.0 -3.113832  0.585864 -5.399730  1.817092 -0.840618 -2.943548   
281674  170348.0  1.991976  0.158476 -2.583441  0.408670  1.151147 -0.096695   

              V7        V8        V9  .

In [9]:
print(Y)

196939    0
182749    0
28340     0
168956    0
137583    0
         ..
279863    1
280143    1
280149    1
281144    1
281674    1
Name: Class, Length: 984, dtype: int64


In [10]:
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.2,stratify=Y,random_state=42)

In [11]:
scaler = StandardScaler()
X_train_standarized = scaler.fit_transform(X_train)
X_test_standarized = scaler.transform(X_test)


In [12]:
X_train_standarized.shape,X_test_standarized.shape

((787, 30), (197, 30))

In [13]:
#training
model = LogisticRegression(max_iter=10000)
model.fit(X_train_standarized,Y_train)


LogisticRegression(max_iter=10000)

In [14]:
#Evaluating
Y_prd = model.predict(X_test_standarized)
accuracy = accuracy_score(Y_test,Y_prd)# used accuracy because the data is balanced
print("The accuracy is ",accuracy)

The accuracy is  0.9543147208121827


#Method 2 . Using the original imbalanced dataset

In [16]:
df.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [17]:
df["Class"].value_counts()

,count
Class,
0,284315
1,492


In [18]:
X = df.drop("Class",axis=1)
Y= df["Class"]


In [20]:
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.2,stratify=Y,random_state=42)

In [21]:
X_train_standarized = scaler.fit_transform(X_train)
X_test_standarized = scaler.transform(X_test)


In [22]:
X_train_standarized.shape,X_test_standarized.shape

((227845, 30), (56962, 30))

In [23]:
#training
model = LogisticRegression(max_iter=10000,class_weight="balanced")
model.fit(X_train_standarized,Y_train)


LogisticRegression(class_weight='balanced', max_iter=10000)

In [24]:
#Evaluating
Y_prd = model.predict(X_test_standarized)
f1_accuracy = f1_score(Y_test,Y_prd)# used f1 because the data is imbalanced
print("The f1 accuracy is ",f1_accuracy)

The f1 accuracy is  0.11435832274459974


#Even though I used class_weight ="balanced" the f1 accuracy is very low which means we need to try another model that is insensitive to imbalanced data. let's try RandomFrostClassifier

In [25]:
from sklearn.ensemble import RandomForestClassifier

model2 = RandomForestClassifier(n_estimators=100, class_weight="balanced", random_state=42)
model2.fit(X_train_standarized, Y_train)
Y_pred_rf = model2.predict(X_test_standarized)
print("F1 Score :", f1_score(Y_test, Y_pred_rf))


F1 Score : 0.8390804597701149


In [31]:
from sklearn.metrics import confusion_matrix, classification_report
print(confusion_matrix(Y_test, Y_pred_rf))
print(classification_report(Y_test, Y_pred_rf))


[[56861     3]
 [   25    73]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     56864
           1       0.96      0.74      0.84        98

    accuracy                           1.00     56962
   macro avg       0.98      0.87      0.92     56962
weighted avg       1.00      1.00      1.00     56962

